Imports

In [19]:
import numpy as np
import pandas as pd
import time
import os
import aux_tools as aux
import Tabesh2013functions as tbsh
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
from collections import defaultdict

### Datos
Se asume que el dataframe tiene una columna para distinguir los clusters de forma global.

In [8]:
mina_df = pd.read_csv("df_mina_clustered_4.csv")
mina_df.rename(columns={'final_cluster_id': 'cluster'}, inplace=True)
mina_df.head()

,x,y,z,au,cpy,cueq,cus,cut,density,material,py,recg_au,recg_cu,tasox,tipomineral,fase,id,banco,cluster,final_cluster_label
0,491425.0,7456205.0,1968.0,0.288929,1.124707,0.712658,0.022003,0.550661,2.615598,8,0.401511,69.001505,87.011938,0.039957,2,4,1,22,1,F4_B22_C1
1,491435.0,7456195.0,1968.0,0.266522,1.057851,0.640223,0.021141,0.491924,2.626056,8,0.197908,68.839492,87.471598,0.042976,2,4,2,22,1,F4_B22_C1
2,491445.0,7456195.0,1968.0,0.243283,2.427782,0.531022,0.019991,0.396348,2.637350,6,0.399009,68.633711,87.659524,0.050438,2,4,3,22,1,F4_B22_C1
3,491455.0,7456195.0,1968.0,0.243283,2.427782,0.531022,0.019991,0.396348,2.637350,6,0.399009,68.633711,87.659524,0.050438,2,4,4,22,1,F4_B22_C1
4,491435.0,7456205.0,1968.0,0.288929,1.124707,0.712658,0.022003,0.550661,2.615598,8,0.401511,69.001505,87.011938,0.039957,2,4,5,22,1,F4_B22_C1


In [3]:
P = 4
R = 0.85 
C_r = 0.25 
C_m = 2 
C_p = 10 
FTL = 2204.62 # Factor tonelada-libra

Para simplificar: Crear un nuevo dataframe con los parametros de cada cluster_label y se necesitan los datos de masa, ley media, fase, banco y z.

In [11]:
mina_df.columns

Index(['x', 'y', 'z', 'au', 'cpy', 'cueq', 'cus', 'cut', 'density', 'material',
       'py', 'recg_au', 'recg_cu', 'tasox', 'tipomineral', 'fase', 'id',
       'banco', 'final_cluster_id', 'final_cluster_label'],
      dtype='object')

In [35]:
vol = 1600  # volumen por bloque, puedes ajustar este valor

# Calcula masa y ley ponderada para cada bloque
mina_df['masa'] = mina_df['density'] * vol
mina_df['fino_total'] = mina_df['masa'] * mina_df['cut']
mina_df['tripleta_fbc'] = list(zip(mina_df['fase'], mina_df['banco'], mina_df['cluster']))
cluster_df = mina_df.groupby('tripleta_fbc').agg({
    'z': 'first',
    'masa': 'sum',
    'fino_total': 'sum'
}).reset_index()

In [36]:
cluster_df

,tripleta_fbc,z,masa,fino_total
0,"(1, 1, 1)",2304.0,57950.030528,655.226403
1,"(1, 1, 2)",2304.0,263780.734000,3732.400225
2,"(1, 1, 3)",2304.0,348369.988064,5151.109044
3,"(1, 1, 4)",2304.0,353305.337520,18069.971643
4,"(1, 1, 5)",2304.0,15277.598544,229.163978
...,...,...,...,...
1296,"(4, 23, 1)",1952.0,322340.933360,177669.295572
1297,"(4, 23, 2)",1952.0,409376.814544,216374.357103
1298,"(4, 23, 3)",1952.0,367240.373488,165874.177477
1299,"(4, 23, 4)",1952.0,359979.638640,172417.045165


In [41]:
params = {
    row['tripleta_fbc']: {col: row[col] for col in cluster_df.columns if col != 'tripleta_fbc'}
    for _, row in cluster_df.iterrows()
}

Ahora falta hacer las dependencias de la restriccion X~Z

In [20]:
arcs = aux.Global_Vertical_Arc_Calculation(mina_df)
# arcs es un diccionario con llave (f,b,c) y guarda una lista de (f_,b_,c_) indicando los cluster superiores
# Convierte arcs a un defaultdict con ints normales
arcs_clean = defaultdict(list)
for key, values in arcs.items():
    # key limpio
    key_clean = tuple(int(k) for k in key)
    # lista de valores limpios
    values_clean = [tuple(int(v) for v in value) for value in values]
    arcs_clean[key_clean] = values_clean

Ejemplo:

In [21]:
arcs[(1,5,1)]

[(np.int64(1), np.int64(4), 1),
 (np.int64(1), np.int64(4), 2),
 (np.int64(1), np.int64(4), 6),
 (np.int64(1), np.int64(4), 8),
 (np.int64(1), np.int64(4), 9),
 (np.int64(1), np.int64(4), 13),
 (np.int64(1), np.int64(4), 15),
 (np.int64(2), np.int64(4), 11)]

In [22]:
arcs_clean[(1,5,1)]

[(1, 4, 1),
 (1, 4, 2),
 (1, 4, 6),
 (1, 4, 8),
 (1, 4, 9),
 (1, 4, 13),
 (1, 4, 15),
 (2, 4, 11)]

Por tanto, de cierta forma se tiene $$ \mathcal{P}_{f,b,c} = arcs[(f,b,c)]$$

IGNORAR el siguiente codigo, no ejecutar

In [33]:
fbc = list(mina_df[['fase', 'banco', 'cluster']].drop_duplicates().itertuples(index=False, name=None))

In [34]:
fbc

[(4, 22, 1),
 (4, 22, 2),
 (4, 23, 1),
 (4, 23, 2),
 (4, 22, 3),
 (4, 22, 4),
 (4, 23, 3),
 (4, 22, 5),
 (4, 23, 4),
 (4, 22, 6),
 (4, 22, 7),
 (4, 22, 8),
 (4, 23, 5),
 (4, 19, 1),
 (4, 20, 1),
 (4, 21, 1),
 (4, 20, 2),
 (3, 21, 1),
 (3, 21, 2),
 (4, 19, 2),
 (3, 22, 1),
 (3, 21, 3),
 (3, 21, 4),
 (4, 21, 2),
 (4, 21, 3),
 (3, 23, 1),
 (3, 23, 2),
 (3, 22, 2),
 (3, 21, 5),
 (4, 19, 3),
 (4, 20, 3),
 (4, 21, 4),
 (3, 22, 3),
 (3, 22, 4),
 (3, 21, 6),
 (4, 20, 4),
 (4, 19, 4),
 (4, 21, 5),
 (3, 23, 3),
 (4, 21, 6),
 (4, 20, 5),
 (4, 19, 5),
 (3, 23, 4),
 (3, 22, 5),
 (3, 21, 7),
 (3, 22, 6),
 (3, 21, 8),
 (4, 21, 7),
 (4, 20, 6),
 (4, 21, 8),
 (4, 21, 9),
 (4, 20, 7),
 (4, 19, 6),
 (4, 20, 8),
 (4, 19, 7),
 (4, 20, 9),
 (4, 19, 8),
 (4, 21, 10),
 (4, 19, 9),
 (4, 19, 10),
 (4, 21, 11),
 (4, 20, 10),
 (4, 19, 11),
 (4, 20, 11),
 (4, 19, 12),
 (4, 16, 1),
 (4, 16, 2),
 (4, 16, 3),
 (3, 18, 1),
 (3, 18, 2),
 (4, 18, 1),
 (3, 19, 1),
 (3, 19, 2),
 (3, 20, 1),
 (3, 19, 3),
 (3, 18, 3),
 (3, 

In [59]:
model = Model("PlanificacionMina")
T = list(range(1, 21)) 
# VARIABLES DE DECISIÓN
y = model.addVars(params.keys(), T, vtype=GRB.BINARY, name="y")
z = model.addVars({(f, b, t) for (f, b, k) in params for t in T}, vtype=GRB.BINARY, name="z")
model.update()
print(len(model.getVars()))


27580


In [ ]:
print(len(model.getVars()))

{(1, 1, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 1, 19): <gurobi.Var *Awaiting

In [ ]:
coord_z = mina[:,2] 
coord_z = [float(h) for h in coord_z] # Pasar a lista
alturas = np.unique(coord_z)
alturas = [int(b) for b in alturas] #pasar a lista
cut = mina[:,7] 
cut = [float(b)/100 for b in cut]
densidad = mina[:,8]
fase = mina[:,15]
fases = np.unique(fase)
fases = [int(f) for f in fases] # Pasar a lista
enumeracion = mina[:,16] 
ley_marginal = C_p/((P-C_r)*FTL*R)
ley_critica = (C_p + C_m)/((P-C_r)*FTL*R)
ley_maxima = 0.004216
P_min = 2 
P_max = 4 
H_diff = 16  
volumen_bloque = 1600 
altura = [[] for _ in range(len(fases))] 
alt = [[] for _ in range(len(fases))]
T = range(1,35)
r = 0.1

In [ ]:
#fase es un array con el valor de la fase correspondiente al bloque
# fases es una lista 1,2,3,..,n_fases
# enumeracion es un array que basicamente es el indice de los bloques
for ID in enumeracion: # for ID en bloques
    for f in range(len(fases)): # for f fase valida
        if fase[int(ID)-1] == fases[f]: # si el bloque tiene la misma fase, agregamos la coordenada z a la lista alt[f] (alturas fase f)
            alt[f].append(coord_z[int(ID)-1])

In [ ]:
for f in range(len(fases)): # for f fase valida
    altura[f] = np.unique(alt[f]) # la lista altura[f] tiene los elementos unicos de alt[f]
P_fb = [[[] for _ in range(len(alturas))] for _ in range(len(fases))]
# Es una lista de listas[f], f fase valids, listas[f] es una lista de |cantidad de alturas distintas en la fase f| listas vacias

for f in range(len(fases)): # f in fases
    for b in range(len(altura[f])): # b in banco?, va de 0 a alturas-1
        # Inicializacion de flags Arriba Lado Abajo
        Condicion_Arriba = True
        Condicion_Lado = True
        Condicion_Abajo = True
        # Check arriba
        if Condicion_Arriba:
            if b+1 < len(altura[f]):
                P_fb[f][b].append([f,b+1]) 
                Condicion_Arriba = False
            for fn in range(len(fases)): 
                for bn in range(len(altura[fn])): 
                    if(fn-f) == 1: 
                        if(altura[fn][bn] - altura[f][b]) == P_max*H_diff: 
                            if Condicion_Lado:
                                P_fb[f][b].append([fn,bn]) 
                                Condicion_Lado = False
        for fn in range(len(fases)):
            for bn in range(len(altura[fn])):
                if(f-fn) == 1: 
                    if(altura[f][b] - altura[fn][bn]) == P_min*H_diff:
                        if Condicion_Abajo:
                            P_fb[f][b].append([fn,bn])
                            Condicion_Abajo = False

In [ ]:
peso_mineral       = [[[[] for _ in range(len(cortes_fb[f][b]))] for b in range(len(altura[f]))] for f in range(len(fases))] 
# peso_mineral[alturas unicas][fases unicas][leyes_unicas] [ lista vacia]
peso_fino          = [[[[] for _ in range(len(cortes_fb[f][b]))] for b in range(len(altura[f]))] for f in range(len(fases))] 

# peso_fino[alturas unicas][fases unicas][leyes_unicas] [lista vacia]




for ID in enumeracion: 
    for f in range(len(fases)): 
        if  f+1 == int(fase[int(ID)-1]): # if fase_bloque = fase f
            for b in range(len(altura[f])): # for b in banco(f) 
                if altura[f][b] == coord_z[int(ID)-1]: # if altura_bloque = altura_f_b   
                     for k in range(len(cortes_fb[f][b])):  # for k in len(leyes_f_b)
                        if cut[int(ID)-1] >= cortes_fb[f][b][k]: # Sí la ley del bloque es mayor a la ley_k
                            peso_mineral[f][b][k].append(float(densidad[int(ID)-1])*volumen_bloque) # Agregar el peso a la lista
                            peso_fino[f][b][k].append(float(densidad[int(ID)-1])*volumen_bloque*float(cut[int(ID-1)]))  # Agregar su peso fino correspondiente

sum_peso_mineral = [[[0 for _ in range(len(cortes_fb[f][b]))] for b in range(len(altura[f]))] for f in range(len(fases))] 
sum_peso_fino = [[[0 for _ in range(len(cortes_fb[f][b]))] for b in range(len(altura[f]))] for f in range(len(fases))] 
for f in range(len(fases)): 
    for b in range(len(altura[f])): # For (f,b)  
        for k in range(len(cortes_fb[f][b])): # for k en leyes fb
            # Se calcula la suma correspondiente
            sum_peso_mineral[f][b][k] = sum(peso_mineral[f][b][k]) #Suma de pesos
            sum_peso_fino[f][b][k] = sum(peso_fino[f][b][k]) #Peso fino


In [ ]:
M = {}
ep = {}
mu = {}
h = {}

for f in range(len(fases)): 
    for b in range(len(altura[f])): # For (f,b)  
        for k in range(len(cortes_fb[f][b])): # for k en leyes fb
            if sum_peso_mineral[f][b][k] != 0: 
                M[f,b,k] = sum_peso_mineral[f][b][k] # Masa con ley corte k
                h[f,b,k] = sum_peso_fino[f][b][k]/sum_peso_mineral[f][b][k] # Ley media

for f in range(len(fases)):
    for b in range(len(altura[f])): # For (f,b)  
        for k in range(len(cortes_fb[f][b])): # for k en leyes fb
            if (f, b, k) in M: 
                if cortes_fb[f][b][k] >  0:
                    ep[f,b,k] = (M[f,b,0]-M[f,b,k])/M[f,b,0] # Complemento
                    mu[f,b,k] = M[f,b,k]/M[f,b,0] #Concentracion
                elif cortes_fb[f][b][k] == 0:
                    ep[f,b,k] = 1
                    mu[f,b,k] = 0  
capacidad_maxima_planta = 5500000
capacidad_maxima_mina = 16000000

In [ ]:
I = {}
for f in range(len(fases)):
    for b in range(len(altura[f])): # For (f,b)  
        for k in range(len(cortes_fb[f][b])): # for k en leyes fb
            if (f, b, k) in M: 
                # (1/(1+r)**t)*((P-C_r)*R*FTL*h[f,b,k]*mu[f,b,k]*x[f,b,t,k]-(C_m+C_p)*mu[f,b,k]*x[f,b,t,k]-C_m*ep[f,b,k]*x[f,b,t,k])
                I[f,b,k] = (P-C_r)*R*FTL*h[f,b,k]*mu[f,b,k]-(C_m+C_p)*mu[f,b,k]-C_m*ep[f,b,k]

$\sum_{t=1}^N \frac{1}{(1+r)^t} \left(\sum_{f,b,k} I_{f,b,k} \hat{y}_{f,b,t,k}\right) x_{f,b,t}$

In [ ]:
for f in range(len(fases)):
    for b in range(len(altura[f])):
        leyes_eliminar = {}
        valores = {}
        for k in range(len(cortes_fb[f][b])):
            if (f,b,k) in M:
                valor = M[f,b,k]
                if valor in valores:
                    leyes_eliminar[f,b,k] = cortes_fb[f][b][k]
                else:
                    valores[valor] = k
            else:
                leyes_eliminar[f,b,k] = cortes_fb[f][b][k]

In [ ]:
fbk_indexes = []
fbtk_indexes = []
for f in range(len(fases)):
    for b in range(len(altura[f])):
        for t in T:
            for k in range(len(cortes_fb[f][b])):
                fbtk_indexes.append((f,b,t,k))
fbt_indexes = []
for f in range(len(fases)):
    for b in range(len(altura[f])):
        for t in T:
            fbt_indexes.append((f,b,t))

In [ ]:
MP = Model()
eta = MP.addVar(name='eta', vtype = GRB.CONTINUOUS)
y = MP.addVars(fbtk_indexes, vtype= GRB.BINARY, name='y')
z = MP.addVars(fbt_indexes, vtype = GRB.BINARY, name='z')
# Agregar restricciones



In [ ]:
for f in range(len(fases)):
    for b in range(len(altura[f])):
        for t in T:
            MP.addConstr(quicksum(y[f,b,t,k] for k in range(len(cortes_fb[f][b]))) <= 1)#3
            if t != 1:
                MP.addConstr(z[f,b,t-1] <= z[f,b,t])#9
            if len(P_fb[f][b]) != 0:
                MP.addConstr(quicksum(z[fn, bn, t] for fn, bn in P_fb[f][b]) / len(P_fb[f][b]) >= quicksum(y[f, b, t, k] for k in range(len(cortes_fb[f][b]))))#8